In [1]:
import pandas as pd
import datetime as dt

from bs4 import BeautifulSoup as bs
import requests
import pymongo
from splinter import Browser

In [2]:
import os
from selenium import webdriver
from selenium.webdriver.chrome.options import Options

In [11]:
options = Options()

# options.binary_location = os.environ.get('GOOGLE_CHROME_BIN')

options.add_argument('--headless')
options.add_argument('--disable-gpu')
options.add_argument('--no-sandbox')
driver = webdriver.Chrome(executable_path='chromedriver.exe', options=options)


In [ ]:
# Set initial url to JPL mars topic content.
url2 ="https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars"
# browser = Browser('chrome', headless=True)
# Visit landing page
browser.get(url2)
# Save HTML to variable




In [ ]:
# # Create BSoup object. Parse HTML

body= browser.find_element_by_tag_name("body").get_attribute('innerHTML')
# body


In [ ]:
jpl = bs(body, 'html.parser')

jpl_base = 'https://www.jpl.nasa.gov'
jpl_link_format = str( jpl_base + (jpl.find_all('footer')[0].a.get('data-link')) )
jpl_link_format

In [5]:
url = 'https://mars.nasa.gov/news/'
browser = Browser('chrome', headless=True) # <<<<<<----- Set headless=False to have browser pop-up
browser.visit(url)
html = browser.html
# Create BeautifulSoup object; parse with 'lxml'
nasa_gov = bs(html, 'lxml')

In [ ]:
nasa_gov

In [6]:
# Returns a list of all the content title, only saving the first found (index [0]) as it is most recent.
news_title = nasa_gov.find_all('div', class_="content_title")[0].get_text()

# Returns a list of all the title body  contents/paragraphs, only saving the first found (index [0]) as it is most recent.

news_content = nasa_gov.find_all('div', class_="article_teaser_body")[0].get_text()

print(news_title)
print(news_content)

Mars 2020 Rover Gets a Super Instrument
With its rock-zapping laser, the SuperCam will enable the science team to identify the chemical and mineral makeup of its targets on the Red Planet.


In [9]:
driver.get('https://mars.nasa.gov/news/')
html = driver.find_element_by_tag_name("body").get_attribute('innerHTML')

# Create BeautifulSoup object; parse with 'lxml'
nasa_gov = bs(html, 'lxml')
# Returns a list of all the content title, only saving the first found (index [0]) as it is most recent.
news_title = nasa_gov.find_all('div', class_="content_title")[0].get_text()
# Returns a list of all the title body  contents/paragraphs, only saving the first found (index [0]) as it is most recent.
news_content = nasa_gov.find_all('div', class_="article_teaser_body")[0].get_text()

In [10]:
print(news_content)
print(news_title)

With its rock-zapping laser, the SuperCam will enable the science team to identify the chemical and mineral makeup of its targets on the Red Planet.
Mars 2020 Rover Gets a Super Instrument


In [ ]:
# Set initial url to JPL mars topic content.
url2 ="https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars"
browser = Browser('chrome', headless=False)
# Visit landing page
browser.visit(url2)
# Save HTML to variable
jpl_html = browser.html

# Create BSoup object. Parse HTML
jpl = bs(jpl_html, 'html.parser')

# jpl

In [ ]:
# Featured image on landing page is not the full size image. 
# Path to the full size image is contained within an <a> tag's href in the footer element.
# This is the concise refactored code
jpl_base = 'https://www.jpl.nasa.gov'
jpl_link_format = str( jpl_base + (jpl.find_all('footer')[0].a.get('data-link')) )

#Vebose method below....

# #Returns the first entry of a list of footer tags
# footer = jpl.find_all('footer')[0].get('data=link')

# # Access the a tag link(.php link) to  use to traverse to that page which contains the full sie image.
# jpl_link = footer.a['data-link']

# # Create the complete link string
# jpl_link_format = str(jpl_base+jpl_link )

jpl_link_format

In [ ]:
#Visit the next page i.e the page where we find the full size image.
browser = Browser('chrome', headless=True)
browser.visit(jpl_link_format)
# Save the html to var
jpl_target = browser.html
# Create new Soup object
jpl_target_html = bs(jpl_target, 'html.parser')

In [ ]:
# Refactored: Find the featued image url. Concise method
featured_img_url = str(jpl_base+ (jpl_target_html.find_all('figure', class_='lede')[0].a.get('href'))  )



#######Verbose method of above

# Find the featued image url. Contained in <figure> , list returned contains one entry but it must be out of list format
# to access inner <a> href.. Use index [0]
# featured_loc = jpl_target_html.find_all('figure', class_='lede')[0]
# featured_url_partial = featured_loc.a['href']

# ## If we didnt use index [0] in the prior cell above^^^ this is the alternate method of accessing the href
# #featured_url_partial = featured_loc[0].a['href']

# # Create complete url string
# featured_img_url = str(jpl_base+featured_url_partial)


featured_img_url

In [ ]:
#  All current required  scraped info and variables up to this cell
print(news_title)
print(news_content)
print(featured_img_url)

In [ ]:
mars_twit="https://twitter.com/marswxreport?lang=en"
browser = Browser('chrome', headless=False)
#Visit the page 
browser.visit(mars_twit)
# Save browsed page to var
mars_browse = browser.html
# Parse HTML
mars_twit_html = bs(mars_browse,'html.parser')
mars_twit_html

In [ ]:
# Returns a list of all <P> tags wityh class 'tweet-text'. This is where the tweets written content is contained.
mars_tweet = mars_twit_html.find_all('p', class_='tweet-text')[0].text

mars_tweet

In [ ]:
# Tweet <p> text is returned with '\n' newline special chatacter. .r/l/strip() neither worked. Must use replace.
mars_tweet = mars_tweet.replace('\n',',')
mars_tweet

In [ ]:
print(news_title)
print(news_content)
print(featured_img_url)
print(mars_tweet)

In [ ]:
# Use pandas to read in the tabular data of facts

facts_url = 'http://space-facts.com/mars/'
mars_facts = pd.read_html(facts_url)
mars_facts

In [ ]:
# Reconstruct mars_facts DF(result of the scrape), to a dataframe that doesn't require list indexing.
mars_facts_pd = pd.DataFrame(mars_facts[0])
mars_facts_pd

In [ ]:
# Rename columns
mars_facts_pd = mars_facts_pd.rename(columns = {0:'Descriptor',1:'Data'})
mars_facts_pd

In [ ]:
mars_facts_html = mars_facts_pd.to_html()
mars_facts_html

In [ ]:
image_site_url = 'https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars'
image_site_base = 'https://astrogeology.usgs.gov'
browser = Browser('chrome', headless=True)
#Visit the page 
browser.visit(image_site_url)
# Save browsed page to var
mars_img_browse = browser.html
# Parse HTML
mars_img_html = bs(mars_img_browse,'html.parser')
mars_img_html

In [ ]:
# Initialize list to store the link to the webpage for an individual hemisphere
mars_hemi_targets = []

# Link contained in a <div> , that has a nested <a>. Use .get() to access the a href. 
# Concat the website base url to the href before appending to list of links used in next traversal.
[mars_hemi_targets.append(image_site_base + div.a.get('href')) for div in mars_img_html.find_all('div', class_='description')]

mars_hemi_targets

###### This is the same as doing the verbose method below.############

# link_locs = mars_img_html.find_all('div', class_='description')

# mars_hemi_links = []
# [mars_hemi_links.append(div.a['href']) for div in link_locs]

# mars_hemi_targets = []
# [mars_hemi_targets.append(image_site_base + link) for link in mars_hemi_links]


In [ ]:
# Create empty list for storing the image title and URL dictionary
#Ex:  {"title": "Valles Marineris Hemisphere", "img_url": "..."},
hemi_title_url = []

#Loop through the 4 targets which correspond to individal pages for each hemisphere. 
# headless=True to execute without browser pop-up

for target in mars_hemi_targets:
    browser = Browser('chrome', headless=True)
    #Visit the page 
    browser.visit(target)
    # Save browsed page to var
    target_obj = browser.html
    # Parse HTML
    target_html= bs(target_obj,'html.parser')
    # Even though the is only one h2 with class='title', find_all() returns the results as a list. Must use index [0].
    title = target_html.find_all('h2', class_='title')[0].text
    # Same logic as above
    hemi_url = target_html.find_all('a', text='Sample')[0].get('href')
    # Append the result in dict format to the empty list created prior
    hemi_title_url.append({'title':title,'img_url':hemi_url})   

In [ ]:
## Results of  the above^^^^^ for loop should match the format of the example below

# hemisphere_image_urls = [
#     {"title": "Valles Marineris Hemisphere", "img_url": "..."},
#     {"title": "Cerberus Hemisphere", "img_url": "..."},
#     {"title": "Schiaparelli Hemisphere", "img_url": "..."},
#     {"title": "Syrtis Major Hemisphere", "img_url": "..."},
# ]
hemi_title_url


In [ ]:
#### Reference of var names and data of all necessary items  scraped, up to this cell

print(news_title)
print(news_content)
print(featured_img_url)
print(mars_tweet)
print(hemi_title_url)
#print(mars_facts_html) <<<------html table. !dont print